In [1]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH17-LangGraph-Use-Cases")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH17-LangGraph-Use-Cases


In [2]:
from langchain_teddynote.models import get_model_name, LLMs

# 모델명 정의
MODEL_NAME = get_model_name(LLMs.GPT4o)
print(MODEL_NAME)

gpt-4o


# 도구 정의

In [3]:
from langchain_teddynote.tools import TavilySearch

# Tavily 검색 도구 초기화
tools = [TavilySearch(max_results=3)]

# 작업 실행 에이전트 정의


In [4]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.prompts import ChatPromptTemplate

# 프롬프트 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer in Korean.",
        ),
        ("human", "{messages}"),
    ]
)


# LLM 정의
llm = ChatOpenAI(model=MODEL_NAME, temperature=0)

# ReAct 에이전트 생성
agent_executor = create_react_agent(llm, tools, state_modifier=prompt)

In [5]:
# 에이전트 실행
agent_executor.invoke(
    {"messages": [("user", "랭체인 한국어 튜토리얼에 대해서 설명해줘")]}
)

{'messages': [HumanMessage(content='랭체인 한국어 튜토리얼에 대해서 설명해줘', additional_kwargs={}, response_metadata={}, id='88d094e4-fdf8-4a2b-ab43-24a033ad9789'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_RET6frawe4cNUfaxgLEPlIKD', 'function': {'arguments': '{"query":"랭체인 한국어 튜토리얼"}', 'name': 'tavily_web_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 154, 'total_tokens': 182, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_90122d973c', 'id': 'chatcmpl-BQ6a242CzvVlRV7krkq9mj7Shynv3', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-edb2cdb1-6f88-431e-93da-85a8d5a70505-0', tool_calls=[{'name': 'tavily_web_search', 'args': {'query': '랭체인 한국어 튜토리얼'}, 'id': 'call_RET6frawe4

# 상태 정의


In [6]:
import operator
from typing import Annotated, List, Tuple
from typing_extensions import TypedDict


# 상태 정의
class PlanExecute(TypedDict):
    input: Annotated[str, "User's input"]
    plan: Annotated[List[str], "Current plan"]
    past_steps: Annotated[List[Tuple], operator.add]
    response: Annotated[str, "Final response"]

# 계획(Plan) 단계

In [7]:
from pydantic import BaseModel, Field
from typing import List


# Plan 모델 정의
class Plan(BaseModel):
    """Sorted steps to execute the plan"""

    steps: Annotated[List[str], "Different steps to follow, should be in sorted order"]

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# 계획 수립을 위한 프롬프트 템플릿 생성
planner_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """For the given objective, come up with a simple step by step plan. \
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.
Answer in Korean.""",
        ),
        ("placeholder", "{messages}"),
    ]
)

planner = planner_prompt | ChatOpenAI(
    model=MODEL_NAME, temperature=0
).with_structured_output(Plan)

In [9]:
## Planner 실행
planner.invoke(
    {
        "messages": [
            (
                "user",
                "LangGraph 의 핵심 장단점과 LangGraph 를 사용하는 이유는 무엇인가?",
            )
        ]
    }
)

Plan(steps=['LangGraph의 정의와 목적을 이해한다.', 'LangGraph의 주요 장점을 나열한다.', 'LangGraph의 주요 단점을 나열한다.', 'LangGraph를 사용하는 이유를 설명한다.'])

# 재계획(Re-Plan) 단계

In [10]:
from typing import Union


class Response(BaseModel):
    """Response to user."""

    # 사용자 응답
    response: str


class Act(BaseModel):
    """Action to perform."""

    # 수행할 작업: "Response", "Plan". 사용자에게 응답할 경우 Response 사용, 추가 도구 사용이 필요할 경우 Plan 사용
    action: Union[Response, Plan] = Field(
        description="Action to perform. If you want to respond to user, use Response. "
        "If you need to further use tools to get the answer, use Plan."
    )


# 계획을 재수립하기 위한 프롬프트 정의
replanner_prompt = ChatPromptTemplate.from_template(
    """For the given objective, come up with a simple step by step plan. \
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.

Your objective was this:
{input}

Your original plan was this:
{plan}

You have currently done the follow steps:
{past_steps}

Update your plan accordingly. If no more steps are needed and you can return to the user, then respond with that. Otherwise, fill out the plan. Only add steps to the plan that still NEED to be done. Do not return previously done steps as part of the plan.

Answer in Korean."""
)


# Replanner 생성
replanner = replanner_prompt | ChatOpenAI(
    model=MODEL_NAME, temperature=0
).with_structured_output(Act)

# 그래프 생성


In [11]:
from langchain_core.output_parsers import StrOutputParser


# 사용자 입력을 기반으로 계획을 생성하고 반환
def plan_step(state: PlanExecute):
    plan = planner.invoke({"messages": [("user", state["input"])]})
    # 생성된 계획의 단계 리스트 반환
    return {"plan": plan.steps}


# 에이전트 실행기를 사용하여 주어진 작업을 수행하고 결과를 반환
def execute_step(state: PlanExecute):
    plan = state["plan"]
    # 계획을 문자열로 변환하여 각 단계에 번호를 매김
    plan_str = "\n".join(f"{i+1}. {step}" for i, step in enumerate(plan))
    task = plan[0]
    # 현재 실행할 작업을 포맷팅하여 에이전트에 전달
    task_formatted = f"""For the following plan:
{plan_str}\n\nYou are tasked with executing [step 1. {task}]."""
    # 에이전트 실행기를 통해 작업 수행 및 결과 수신
    agent_response = agent_executor.invoke({"messages": [("user", task_formatted)]})
    # 이전 단계와 그 결과를 포함하는 딕셔너리 반환
    return {
        "past_steps": [(task, agent_response["messages"][-1].content)],
    }


# 이전 단계의 결과를 바탕으로 계획을 업데이트하거나 최종 응답을 반환
def replan_step(state: PlanExecute):
    output = replanner.invoke(state)
    # 응답이 사용자에게 반환될 경우
    if isinstance(output.action, Response):
        return {"response": output.action.response}
    # 추가 단계가 필요할 경우 계획의 단계 리스트 반환
    else:
        next_plan = output.action.steps
        if len(next_plan) == 0:
            return {"response": "No more steps needed."}
        else:
            return {"plan": next_plan}


# 에이전트의 실행 종료 여부를 결정하는 함수
def should_end(state: PlanExecute):
    if "response" in state and state["response"]:
        return "final_report"
    else:
        return "execute"


final_report_prompt = ChatPromptTemplate.from_template(
    """You are given the objective and the previously done steps. Your task is to generate a final report in markdown format.
Final report should be written in professional tone.

Your objective was this:

{input}

Your previously done steps(question and answer pairs):

{past_steps}

Generate a final report in markdown format. Write your response in Korean."""
)

final_report = (
    final_report_prompt
    | ChatOpenAI(model=MODEL_NAME, temperature=0)
    | StrOutputParser()
)


def generate_final_report(state: PlanExecute):
    past_steps = "\n\n".join(
        [
            f"Question: {past_step[0]}\n\nAnswer: {past_step[1]}\n\n####"
            for past_step in state["past_steps"]
        ]
    )
    response = final_report.invoke({"input": state["input"], "past_steps": past_steps})
    return {"response": response}

# 그래프 생성

In [12]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver


# 그래프 생성
workflow = StateGraph(PlanExecute)

# 노드 정의
workflow.add_node("planner", plan_step)
workflow.add_node("execute", execute_step)
workflow.add_node("replan", replan_step)
workflow.add_node("final_report", generate_final_report)

# 엣지 정의
workflow.add_edge(START, "planner")
workflow.add_edge("planner", "execute")
workflow.add_edge("execute", "replan")
workflow.add_edge("final_report", END)

# 조건부 엣지: replan 후 종료 여부를 결정하는 함수 사용
workflow.add_conditional_edges(
    "replan",
    should_end,
    {"execute": "execute", "final_report": "final_report"},
)

# 그래프 컴파일
app = workflow.compile(checkpointer=MemorySaver())

# 그래프 실행


In [13]:
from langchain_teddynote.messages import invoke_graph, random_uuid
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(recursion_limit=50, configurable={"thread_id": random_uuid()})

inputs = {
    "input": "Modular RAG 가 기존의 Naive RAG 와 어떤 차이가 있는지와 production level 에서 사용하는 이점을 설명해줘"
}

invoke_graph(app, inputs, config)


🔄 Node: planner 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
Modular RAG와 Naive RAG의 정의를 각각 설명한다.
Modular RAG의 구조적 특징을 설명한다.
Naive RAG의 구조적 특징을 설명한다.
Modular RAG가 Naive RAG와 비교하여 가지는 장점을 설명한다.
Production level에서 Modular RAG를 사용할 때의 이점을 설명한다.

🔄 Node: agent in [execute] 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
================================== Ai Message ==================================

1. **Modular RAG (Retrieval-Augmented Generation)**:  
   Modular RAG는 정보 검색과 생성 모델을 결합하여 정보를 생성하는 시스템입니다. 이 시스템은 검색 모듈과 생성 모듈로 구성되어 있으며, 검색 모듈은 외부 데이터베이스에서 관련 정보를 검색하고, 생성 모듈은 검색된 정보를 바탕으로 자연어 응답을 생성합니다. 이러한 모듈화된 접근 방식은 각 모듈을 독립적으로 최적화할 수 있는 유연성을 제공합니다.

2. **Naive RAG (Retrieval-Augmented Generation)**:  
   Naive RAG는 기본적인 형태의 RAG 시스템으로, 검색과 생성 과정을 단순히 결합한 형태입니다. 이 시스템은 검색된 정보를 그대로 생성 모듈에 전달하여 응답을 생성합니다. 구조적으로 단순하며, 모듈 간의 상호작용이 깊이 있게 설계되지 않은 것이 특징입니다. 

이 두 가지 정의는 RAG 시스템의 기본적인 개념을 이해하는 데 도움을 줍니다.

🔄 Node: execute 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
('Modular 

In [15]:
snapshot = app.get_state(config).values
print(snapshot["response"])


# Modular RAG와 Naive RAG 비교 및 Production Level에서의 이점

## 서론

Retrieval-Augmented Generation(RAG)은 정보 검색과 자연어 생성 모델을 결합하여 사용자에게 보다 정확하고 관련성 높은 정보를 제공하는 시스템입니다. RAG는 크게 Modular RAG와 Naive RAG로 구분할 수 있으며, 이 두 가지 접근 방식은 구조적 차이와 그에 따른 장점을 가지고 있습니다. 본 보고서는 Modular RAG와 Naive RAG의 차이점과 Modular RAG가 Production 환경에서 가지는 이점을 설명합니다.

## Modular RAG와 Naive RAG의 정의

### Modular RAG

Modular RAG는 정보 검색과 생성 모델을 모듈화하여 결합한 시스템입니다. 이 시스템은 검색 모듈과 생성 모듈로 구성되어 있으며, 각 모듈은 독립적으로 최적화가 가능합니다. 이러한 모듈화된 접근 방식은 시스템의 유연성을 높이고, 다양한 요구사항에 맞게 조정할 수 있는 장점을 제공합니다.

### Naive RAG

Naive RAG는 기본적인 형태의 RAG 시스템으로, 검색과 생성 과정을 단순히 결합한 구조입니다. 검색된 정보를 그대로 생성 모듈에 전달하여 응답을 생성하며, 구조적으로 단순한 것이 특징입니다. 모듈 간의 상호작용이 깊이 있게 설계되지 않았습니다.

## Modular RAG의 구조적 특징

1. **모듈화된 아키텍처**: 독립적인 모듈로 구성되어 각 모듈을 독립적으로 개발 및 개선할 수 있습니다.
2. **정보 검색 모듈**: 외부 데이터베이스에서 관련 정보를 검색하는 역할을 합니다.
3. **생성 모듈**: 검색된 정보를 바탕으로 자연어 응답을 생성합니다.
4. **통합 및 조정**: 각 모듈 간의 효과적인 통합과 조정이 중요합니다.
5. **확장성 및 유지보수 용이성**: 모듈화된 구조 덕분에 시스템의 확장성과 유지보수가 용이합니다.

## Naive RAG의 구조적 특징

1. 

In [16]:
from IPython.display import Markdown

Markdown(snapshot["response"])

# Modular RAG와 Naive RAG 비교 및 Production Level에서의 이점

## 서론

Retrieval-Augmented Generation(RAG)은 정보 검색과 자연어 생성 모델을 결합하여 사용자에게 보다 정확하고 관련성 높은 정보를 제공하는 시스템입니다. RAG는 크게 Modular RAG와 Naive RAG로 구분할 수 있으며, 이 두 가지 접근 방식은 구조적 차이와 그에 따른 장점을 가지고 있습니다. 본 보고서는 Modular RAG와 Naive RAG의 차이점과 Modular RAG가 Production 환경에서 가지는 이점을 설명합니다.

## Modular RAG와 Naive RAG의 정의

### Modular RAG

Modular RAG는 정보 검색과 생성 모델을 모듈화하여 결합한 시스템입니다. 이 시스템은 검색 모듈과 생성 모듈로 구성되어 있으며, 각 모듈은 독립적으로 최적화가 가능합니다. 이러한 모듈화된 접근 방식은 시스템의 유연성을 높이고, 다양한 요구사항에 맞게 조정할 수 있는 장점을 제공합니다.

### Naive RAG

Naive RAG는 기본적인 형태의 RAG 시스템으로, 검색과 생성 과정을 단순히 결합한 구조입니다. 검색된 정보를 그대로 생성 모듈에 전달하여 응답을 생성하며, 구조적으로 단순한 것이 특징입니다. 모듈 간의 상호작용이 깊이 있게 설계되지 않았습니다.

## Modular RAG의 구조적 특징

1. **모듈화된 아키텍처**: 독립적인 모듈로 구성되어 각 모듈을 독립적으로 개발 및 개선할 수 있습니다.
2. **정보 검색 모듈**: 외부 데이터베이스에서 관련 정보를 검색하는 역할을 합니다.
3. **생성 모듈**: 검색된 정보를 바탕으로 자연어 응답을 생성합니다.
4. **통합 및 조정**: 각 모듈 간의 효과적인 통합과 조정이 중요합니다.
5. **확장성 및 유지보수 용이성**: 모듈화된 구조 덕분에 시스템의 확장성과 유지보수가 용이합니다.

## Naive RAG의 구조적 특징

1. **정보 검색 단계**: 질문에 관련된 문서를 검색합니다.
2. **문서 선택**: 검색된 문서 중 가장 관련성이 높은 문서를 선택합니다.
3. **생성 모델**: 선택된 문서를 바탕으로 답변을 생성합니다.
4. **결합된 학습**: 검색과 생성을 결합하여 학습할 수 있습니다.

## Modular RAG의 장점

1. **유연성**: 다양한 모듈을 조합하여 시스템을 쉽게 조정할 수 있습니다.
2. **확장성**: 새로운 기능이나 데이터 소스를 추가할 때 용이합니다.
3. **성능 최적화**: 각 모듈을 독립적으로 최적화할 수 있습니다.
4. **재사용성**: 특정 모듈을 다른 프로젝트나 시스템에서 재사용할 수 있습니다.
5. **유지보수 용이성**: 모듈별로 문제를 진단하고 수정할 수 있습니다.

## Production Level에서 Modular RAG의 이점

1. **확장성**: 대규모 데이터 처리 및 다양한 요구 사항에 유연하게 대응할 수 있습니다.
2. **유연한 아키텍처**: 각 구성 요소를 독립적으로 개발, 테스트 및 배포할 수 있습니다.
3. **효율적인 정보 검색**: 더 정확하고 관련성 높은 정보를 제공합니다.
4. **개선된 성능**: 다양한 모듈을 최적화하여 성능을 개선할 수 있습니다.
5. **데이터 보안 및 프라이버시**: 모듈별로 보안 정책을 적용할 수 있습니다.
6. **비용 효율성**: 필요한 모듈만 사용하여 비용을 절감할 수 있습니다.

## 결론

Modular RAG는 Naive RAG에 비해 구조적 유연성과 확장성을 제공하며, Production 환경에서의 다양한 요구사항에 효과적으로 대응할 수 있는 장점을 가지고 있습니다. 이러한 이점들은 Modular RAG를 복잡하고 다양한 환경에서 매력적인 선택지로 만듭니다.